In [17]:
import os
import pandas as pd
import csv
import math
import numpy as np
from ipywidgets import Dropdown

In [64]:
def load_data(file):
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path, delimiter=';', usecols=['GazeTime', 'Region', 'Target', 'PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ'])
    data = data[~((data['PosX'] == 0) & (data['PosY'] == 0) & (data['PosZ'] == 0))]

    # Get the unique regions
    regions = data['Region'].unique()

    # Skip the first region
    for region in regions[1:]:
        # Get the index of the first row of the current region
        first_row_index = data[data['Region'] == region].index[0]
        # Drop the first row of the current region
        data = data.drop(first_row_index)

    # Normalize position data using Min-Max normalization
    pos_columns = ['PosX', 'PosY', 'PosZ', 'PlayerPosX', 'PlayerPosY', 'PlayerPosZ']
    for column in pos_columns:
        data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())

    return data

# Create a list to store the dataframes
dataframes = [load_data(file) for file in os.listdir(folder_path) if file.endswith('.csv')]
dataframes[1]

,GazeTime,Region,Target,PosX,PosY,PosZ,PlayerPosX,PlayerPosY,PlayerPosZ
0,0.127132,1,None,0.073534,0.782909,0.782545,0.000000,0.604793,0.000000
1,0.257327,1,Size(Max),0.029578,0.856803,0.766778,0.000000,0.526312,0.118034
2,0.019057,1,Size(Max),0.000000,0.891201,0.748300,0.000000,0.520501,0.126776
3,0.306683,1,Size(Max),0.010553,0.947649,0.720906,0.000000,0.426968,0.267449
4,0.081326,1,Size(Low),0.029846,0.996627,0.714420,0.000000,0.402166,0.304752
5,0.416918,1,Size(Low),0.060992,1.000000,0.678169,0.000000,0.275011,0.495990
6,0.116016,1,None,0.133364,0.964814,0.669078,0.000000,0.239627,0.549205
7,0.062330,1,None,0.209971,0.944146,0.662564,0.000000,0.220618,0.577796
8,0.077344,1,None,0.287500,0.896645,0.653154,0.000000,0.197030,0.613273
9,0.089137,1,None,0.367703,0.867611,0.644334,0.000000,0.169844,0.654159


In [52]:
def sum(arr):
    ret = 0
    for i in arr:
        ret += i
    return ret

def PercFixInside(dataframe):
    def Compute(region, interest):
        ret = pd.DataFrame()
        target_names = dataframe[dataframe['Region'] == region]['Target'].unique()
        grouped_data = dataframe[dataframe['Region'] == region].groupby('Target')
        gaze = [0] * len(target_names)  # Initialize list with zeros
        perc = [0] * len(target_names)
        ret['Target'] = target_names
        
        for j, target in enumerate(target_names):
            gaze[j] = grouped_data.get_group(target)[interest].sum()
        tot = sum(gaze)

        for i, g in enumerate(gaze):
            perc[i] = (g / tot) * 100 if tot > 0 else 0

        ret['GazeTime'] = gaze
        ret['Percentage'] = perc
        return ret

    regions = dataframe['Region'].unique()

    # Initialize a list to store the DataFrames for each region
    results = []

    for region in regions:
        newdata = Compute(region, 'GazeTime')
        newdata['Region'] = region  # Add a column for the region
        results.append(newdata)

    # Concatenate all the result DataFrames into one
    final_df = pd.concat(results, ignore_index=True)

    return final_df



In [53]:
def NFix(dataframe, threshold, ShowPercentage = False):
    if(threshold < 0):
        threshold = abs(threshold)
    nfix = {}  # Initialize the NFix counter as a dictionary
    counter = {}  # Initialize the counter as a dictionary
    pos = {}  # Variables to track previous position for each target

    for _, row in dataframe.iterrows():
        region = row['Region']
        target = row['Target']
        if region not in counter:
            counter[region] = {}
            nfix[region] = {}
        if target not in counter[region]:
            counter[region][target] = 0
            nfix[region][target] = 0
            pos[region, target] = (None, None)

        counter[region][target] += 1
        pos_x_new, pos_y_new = float(row['PosX']), float(row['PosY'])  # Get the X and Y positions

        # Check if a fixation occurred by comparing the current position with the previous position
        if pos[region, target][0] is not None and pos[region, target][1] is not None:
            pos_diff = abs(pos[region, target][0] - pos_x_new) + abs(pos[region, target][1] - pos_y_new)
            if pos_diff > threshold:
                nfix[region][target] += 1  # Increment the NFix counter for the target

        pos[region, target] = (pos_x_new, pos_y_new)  # Update the previous position for the target

    if ShowPercentage:
        for region in nfix:
            for target in nfix[region]:
                nfix[region][target] = (nfix[region][target] / counter[region][target]) * 100 if counter[region][target] > 0 else 0

    # Convert nested dictionary to DataFrame
    nfix_df = pd.DataFrame([(reg, tar, val) for reg, tar_val in nfix.items() for tar, val in tar_val.items()], 
                           columns=['Region', 'Target', 'NFix'])

    return nfix_df

In [61]:
def ConvergTime(dataframe):
    # Group the data by region and then by target
    dataframe = dataframe[dataframe['Target'] != 'None']
    grouped_data = dataframe.groupby(['Region', 'Target'])

    # Initialize an empty DataFrame to store the results
    results = pd.DataFrame(columns=['Region', 'Target', 'ConvergenceTime'])

    # Iterate over each group (region and target)
    for (region, target), group in grouped_data:
        # Calculate the average gaze time for the current group (target within the region)
        avg_gaze_time = group['GazeTime'].mean()
        
        # Append the result to the results DataFrame
        results = pd.concat([results, pd.DataFrame({'Region': [region], 'Target': [target], 'ConvergenceTime': [avg_gaze_time]})], ignore_index=True)

    return results

In [62]:
def calculate_distance(df):
    # Calculate the Euclidean distance
    df['Distance'] = np.sqrt((df['PosX'] - df['PlayerPosX'])**2 + (df['PosY'] - df['PlayerPosY'])**2 + (df['PosZ'] - df['PlayerPosZ'])**2)
    # Remove rows with 'None' Target
    df = df[df['Target'] != 'None']
    # Group by 'Target' and calculate mean distance
    df = df.groupby('Target')['Distance'].mean().reset_index()
    return df



In [56]:
gaze_data = dataframes[2]
#PercFixInside(gaze_data)
#NFix(gaze_data, 0.05, False)
#ConvergTime(gaze_data)

In [57]:
# Step 1: Apply PercFixInside to each DataFrame in dataframes and accumulate the results
perc_results = [PercFixInside(df) for df in dataframes]

# Step 2: Concatenate the resulting DataFrames into one
all_perc = pd.concat(perc_results)

# Step 3: Group by 'Region' and 'Target' and calculate the mean
avg_perc = all_perc.groupby(['Region', 'Target']).mean().reset_index()

# This will now be a DataFrame with average 'GazeTime' and 'Percentage' for each 'Target' in each 'Region'
avg_perc

,Region,Target,GazeTime,Percentage
0,1,None,1.019422,52.015944
1,1,Size(Low),0.498244,23.611671
2,1,Size(Max),0.561987,29.383651
3,1,Size(Mid),0.457346,23.512021
4,1,Size(Min),0.125721,6.834713
5,2,None,0.778381,41.783133
6,2,Transparency(Low),0.452334,23.168532
7,2,Transparency(Max),0.477530,26.284338
8,2,Transparency(Mid),0.387664,20.971665
9,2,Transparency(Min),0.325784,17.999199


In [58]:
# Step 1: Apply NFix to each DataFrame in dataframes and accumulate the results
nfix_results = [NFix(df, 0.05, False) for df in dataframes]

# Step 2: Concatenate the resulting DataFrames into one
all_nfix = pd.concat(nfix_results)

# Step 3: Group by 'Region' and 'Target' and calculate the mean
avg_nfix = all_nfix.groupby(['Region', 'Target']).mean().reset_index()

# This will now be a DataFrame with average 'Count' and 'Percentage' for each 'Target' in each 'Region'
avg_nfix

,Region,Target,NFix
0,1,None,10.000000
1,1,Size(Low),0.000000
2,1,Size(Max),2.000000
3,1,Size(Mid),1.666667
4,1,Size(Min),0.000000
5,2,None,7.000000
6,2,Transparency(Low),1.000000
7,2,Transparency(Max),0.500000
8,2,Transparency(Mid),1.666667
9,2,Transparency(Min),1.500000


In [59]:
def overall_average_distance(dataframes):
    # Create a list to store the distance dataframes
    distance_dfs = []

    # Loop through each dataframe in dataframes
    for df in dataframes:
        # Calculate the distance for each Target in the dataframe
        distance_df = calculate_distance(df)
        # Append the resulting dataframe to the list
        distance_dfs.append(distance_df)

    # Concatenate all distance dataframes into a single dataframe
    all_data = pd.concat(distance_dfs)

    # Group by Target and calculate the average distance
    all_data = all_data.groupby('Target')['Distance'].mean().reset_index()

    return all_data

# usage
overall_avg_distances = overall_average_distance(dataframes)
overall_avg_distances

,Target,Distance
0,Black,1.002939
1,Blue,0.981945
2,Red,1.265058
3,Size(Low),0.736290
4,Size(Max),0.907025
5,Size(Mid),1.005827
6,Size(Min),0.875614
7,Transparency(Low),0.226945
8,Transparency(Max),0.720127
9,Transparency(Mid),0.523582


In [63]:
def overall_average_convergence_time(dataframes):
    # Create a list to store the convergence time dataframes
    convergence_time_dfs = []

    # Loop through each dataframe in dataframes
    for df in dataframes:
        # Calculate the convergence time for each 'Region' and 'Target' in the dataframe
        convergence_time_df = ConvergTime(df)
        # Append the resulting dataframe to the list
        convergence_time_dfs.append(convergence_time_df)

    # Concatenate all convergence time dataframes into a single dataframe
    all_data = pd.concat(convergence_time_dfs)

    # Group by 'Region' and 'Target' and calculate the average convergence time
    all_data = all_data.groupby(['Region', 'Target'])['ConvergenceTime'].mean().reset_index()

    return all_data

# usage
overall_avg_convergence_times = overall_average_convergence_time(dataframes)
overall_avg_convergence_times

,Region,Target,ConvergenceTime
0,1,Size(Low),0.249122
1,1,Size(Max),0.140953
2,1,Size(Mid),0.173567
3,1,Size(Min),0.125721
4,2,Transparency(Low),0.226167
5,2,Transparency(Max),0.238765
6,2,Transparency(Mid),0.114612
7,2,Transparency(Min),0.127607
8,3,Black,0.227091
9,3,Blue,0.306919


In [78]:
import numpy as np

def calculate_eccentricity(dataframe):
    # Get vectors from the eye to the object and from the eye straight ahead
    dataframe = dataframe.copy()
    dataframe = dataframe[dataframe['Target'] != 'None']

    eye_to_object = dataframe[['PosX', 'PosY', 'PosZ']].values - dataframe[['PlayerPosX', 'PlayerPosY', 'PlayerPosZ']].values
    straight_ahead = np.array([0, 0, 1])  # This assumes that straight ahead is along the z-axis

    # Normalize the vectors (make them length 1) so that dot product gives the cosine of the angle
    eye_to_object = eye_to_object / np.linalg.norm(eye_to_object, axis=1, keepdims=True)
    straight_ahead = straight_ahead / np.linalg.norm(straight_ahead)

    # Calculate the dot product of the two vectors, which is equal to the cosine of the angle between them
    cos_angle = np.dot(eye_to_object, straight_ahead)

    # Calculate the angle itself (in radians)
    angle = np.arccos(cos_angle)

    # Convert to degrees
    angle = np.degrees(angle)

    # Normalize the angle by the Oculus Quest 2's field of view
    fov = 89
    angle = angle / fov

    # Add the angle to the dataframe
    dataframe['Eccentricity'] = angle

    return dataframe

calculate_eccentricity(dataframes[1])


,GazeTime,Region,Target,PosX,PosY,PosZ,PlayerPosX,PlayerPosY,PlayerPosZ,Eccentricity
1,0.257327,1,Size(Max),0.029578,0.856803,0.766778,0.000000,0.526312,0.118034,0.304363
2,0.019057,1,Size(Max),0.000000,0.891201,0.748300,0.000000,0.520501,0.126776,0.346218
3,0.306683,1,Size(Max),0.010553,0.947649,0.720906,0.000000,0.426968,0.267449,0.550039
4,0.081326,1,Size(Low),0.029846,0.996627,0.714420,0.000000,0.402166,0.304752,0.623161
5,0.416918,1,Size(Low),0.060992,1.000000,0.678169,0.000000,0.275011,0.495990,0.853282
10,0.136012,1,Size(Mid),0.462937,0.814328,0.632759,0.000000,0.128362,0.716547,1.076194
11,0.228901,1,Size(Mid),0.445510,0.820428,0.616130,0.000000,0.058550,0.821542,1.158448
12,0.191975,1,Size(Mid),0.419423,0.823219,0.598645,0.000000,0.000000,0.909600,1.220240
14,0.040515,2,Transparency(Max),0.105750,0.857131,0.001224,0.494496,0.982382,1.000000,1.772574
16,0.308692,2,Transparency(Max),0.165234,0.940536,0.055546,0.495543,0.985222,0.780069,1.744889
